In [1]:
# [CELL ID] 1

import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

[CELL ID] 2
## From adversarial examples to training robust models

In the previous notebooks, we focused on methods for solving the maximization problem over perturbations; that is, to finding the solution to the problem
\begin{equation}
\DeclareMathOperator*{\maximize}{maximize}
\maximize_{\|\delta\| \leq \epsilon} \ell(h_\theta(x + \delta), y).
\end{equation}

In this notebook, we will focus on training a robust classifier. More precisly, we aim at solving following minimization problem, namely Adversarial Training:
\begin{equation}
\DeclareMathOperator*{\minimize}{minimize}
\minimize_\theta \frac{1}{|S|} \sum_{x,y \in S} \max_{\|\delta\| \leq \epsilon} \ell(h_\theta(x + \delta), y).
\end{equation}
The order of the min-max operations is important here.  Specially, the max is inside the minimization, meaning that the adversary (trying to maximize the loss) gets to "move" _second_.  We assume, essentially, that the adversary has full knowledge of the classifier parameters $\theta$, and that they get to specialize their attack to whatever parameters we have chosen in the outer minimization. The goal of the robust optimization formulation, therefore, is to ensure that the model cannot be attacked _even if_ the adversary has full knowledge of the model.  Of course, in practice we may want to make assumptions about the power of the adversary but it can be difficult to pin down a precise definition of what we mean by the "power" of the adversary, so extra care should be taken in evaluating models against possible "realistic" adversaries.

## Exercice 1
1. Train a robust classifier using Adversarial Training
2. Evaluate your classifier on natural and adversarial examples
3. Make an analysis and conclude

## Loading MNIST dataset (train set and test set)

In [2]:
# [CELL ID] 3

# load MNIST dataset
def load_mnist(split, batch_size):
  train = True if split == 'train' else False
  dataset = datasets.MNIST("./data", train=split, download=True, transform=transforms.ToTensor())
  return DataLoader(dataset, batch_size=batch_size, shuffle=train)

batch_size = 100
train_loader = load_mnist('train', batch_size)
test_loader = load_mnist('test', batch_size)

## Implementing FGSM and PGD

Nous reprenons le codage des classes réalisé dans le Notebook précédent.

In [3]:
# [CELL ID] 4

class FastGradientSignMethod:
  def __init__(self, model, eps):
    self.model = model
    self.eps = eps
  
  def compute(self, x, y):
    """ Construct FGSM adversarial perturbation for examples x"""    
    x.requires_grad=True # enable locally gradient computation on x
    
    output = self.model(x)
    loss_func = nn.CrossEntropyLoss() # define a specific loss to compute the gradient w.r.t x
    loss = loss_func(output, y)
    loss.backward() # back-propagate the gradient w.r.t x

    delta = self.eps * x.grad.sign() # compute the delta of pertubation by applying the sign of the gradient of x
    x.requires_grad=False # disable gradient computation on x
    
    return x + delta # return the attacked (modified) image


class ProjectedGradientDescent:
  
  def __init__(self, model, eps, alpha, num_iter):
    self.model = model
    self.eps = eps
    self.alpha = alpha
    self.num_iter = num_iter
  
  def compute(self, x, y):
    # we define a specific loss to compute the gradient w.r.t x
    loss_func = nn.CrossEntropyLoss()
    
    # then initialize the delta to 0 with the same shape as x
    delta = torch.zeros_like(x, requires_grad=True)

    # and perform gradient descent iterative procedure
    for i in range(self.num_iter):
      # we compute the output image from the modified image
      output = self.model(x + delta)

      # evaluate the loss on the outup
      loss = loss_func(output, y)
      
      # back-propagate the gradient w.r.t delta
      loss.backward()

      # update the delta with it current gradient
      delta.data += self.alpha * delta.grad.data

      # clip the delta in the range [-eps, eps]
      delta.data = delta.data.clamp(-self.eps, self.eps)

      # reset the gradient on delta
      delta.grad.zero_()

    return x + delta.detach() # return the modified (attacked) image


## Implementing convolutional network

Nous reprenons également le codage du modèle convolutif réalisé dans le Notebook précédent.

In [4]:
# [CELL ID] 5

# define a neural network with 3 fully connected layers
class FullyConnectedModel(torch.nn.Module):
  
  def __init__(self, input_dim, output_dim):
    super(FullyConnectedModel, self).__init__()
    self.layers = nn.Sequential(nn.Flatten(), 
                                nn.Linear(input_dim, 256), nn.ReLU(),
                                nn.Linear(256, 64), nn.ReLU(),
                                nn.Linear(64, output_dim)
    )
        
  def forward(self, x):
    return self.layers(x)
    
# convolutional model with 2 convolution, 2 max pooling layer, 3 fully connected layer
# the model should be: 2x (conv -> max pooling -> relu) -> 2x (fc -> relu) -> fc
class ConvModel(torch.nn.Module):
    def __init__(self):
        super(ConvModel, self).__init__()
        
        # 2 convolutional layers + 3 fully connected layers
        self.layers = nn.Sequential(
            # 1st conv layer
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=3, stride=1, padding=1), # -> input=8, output=28
            nn.MaxPool2d(kernel_size=2), nn.ReLU(), # => output = 28/2 = 14
            
            #2nd conv layer
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=3, stride=1, padding=1), # input=14, output=14
            nn.MaxPool2d(kernel_size=2), nn.ReLU(), # => output = 14/2 = 7
            
            # we use the fully connected layers defined above
            FullyConnectedModel(16*7*7, 10)
        ) 
    
    def forward(self, x):
        return self.layers(x)

## Adversarial training on train set

La fonction `adversarial_train_model` reprend pour l'essentiel la fonction d'entraînement `train_model` développée dans le Notebook 2. La seule différence réside dans le fait que la nouvelle fonction nécessite un paramètre supplémentaire `attack` permettant de lui passer le type d'attaque réalisée. Ceci ce traduit dans le corps de la fonction par la présence, en cas d'attaque :
- d'une nouvelle instruction `delta = attack.compute(imgs, labels)` destinée à récupérer le tenseur des perturbations causées par l'attaque ;
- d'une instruction `imgs = imgs + delta` qui ajoute à l'image d'origine les valeurs des perturbations.

C'est sur la base d'une image perturbée (ou pas si le paramètre `attack`n'est pas renseigné) qu'est ensuite réalisé l'entraînement du modèle. En cas d'attaque, comme précisé dans l'énoncé, l'ordre des opérations est le suivant :

1. L'attaquant prend connaissance du vecteur de paramètres $\theta$ (attaque de type white box) ;
2. Connaissant $\theta$, il en déduit la valeur de $\delta$ qui permet de maximiser la loss, dans les limites fixées par le paramètre $\epsilon$ ;
3. Sachant le $\delta$ utilisé par l'attaquant, la défense consiste à ajuster le vecteur de paramètres $\theta$ de façon à minimiser la loss.

Le déroulé du programme qui en découle est le suivant :
1. Calcul `delta = attack.compute(imgs, labels)` du $\delta$ utilisé par l'attaquant ;
2. Perturbation `imgs = imgs + delta` de l'image initiale découlant de la valeur de $\delta$ utilisée ;
3. Apprentissage sur l'image bruitée (lignes `optimizer.zero_grad()` à `optimizer.step()`) pour déterminer le nouveau vecteur de paramètres $\theta$ optimal après attaque.

In [5]:
# [CELL ID] 6

def adversarial_train_model(model, criterion, optimizer, loader, attack):
  """
    Function to train the model in adverserial way. The training is quiet different from the standard one.
    In fact, the optimisation we'll be performed on the attacked image.
    Therefore, we have 2 steps in our optimisation procedure:
      - compute the adverserial image (Max)
      - optimize the objective function (loss) on the adversarial image (Min)
  """
  losses = []
  for epoch in range(10):
    for imgs, labels in loader:
      imgs, labels = imgs.to(device), labels.to(device)
      
      # in case attack is provided, we modify (MAX step) the original before optimizing the bjective
      if attack:
        imgs = attack.compute(imgs, labels)

      # performing the optimization on the potential modified image (MIN step)
      output = model(imgs)
      loss = criterion(output, labels)

      if optimizer:
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
      
      # backup the loss for furture analysis
      losses.append(loss.detach().cpu().numpy())

    print('epoch {}, loss: {:.4f}'.format(epoch, losses[-1]))

#### Training parameters

In [6]:
# define your loss
criterion = torch.nn.CrossEntropyLoss()

# epsilon
epsilon = 0.1

### Training convolution network under FGSM

In [7]:
# we define the conv model trained under FGSM
fgsm_model = ConvModel().to(device)

# define the optimizer for FGSM training
fgsm_optimizer = torch.optim.SGD(fgsm_model.parameters(), lr=0.1)

# define FGSM attack object
fgsm_attack = FastGradientSignMethod(fgsm_model, epsilon)

# train our model with FGSM perturbation
adversarial_train_model(fgsm_model, criterion, fgsm_optimizer, train_loader, fgsm_attack)

epoch 0, loss: 0.4265
epoch 1, loss: 0.2300
epoch 2, loss: 0.4367
epoch 3, loss: 0.1628
epoch 4, loss: 0.2528
epoch 5, loss: 0.1967
epoch 6, loss: 0.0939
epoch 7, loss: 0.1131
epoch 8, loss: 0.1238
epoch 9, loss: 0.1257


### Training convolution network under PGD

In [8]:
# we define PGD the hyperparameters
alpha = 1e-2
num_iter = 5

# we define the conv model trained under PGD
pgd_model = ConvModel().to(device)

# define the optimizer for FGSM training
pgd_optimizer = torch.optim.SGD(pgd_model.parameters(), lr=0.1)

# define PGD attack object
pgd_attack = ProjectedGradientDescent(model=pgd_model, eps=epsilon, alpha=alpha, num_iter=num_iter )

# train our model with PGD perturbation
adversarial_train_model(pgd_model, criterion, pgd_optimizer, train_loader, pgd_attack)

epoch 0, loss: 0.2651
epoch 1, loss: 0.2035
epoch 2, loss: 0.0961
epoch 3, loss: 0.1013
epoch 4, loss: 0.1465
epoch 5, loss: 0.0897
epoch 6, loss: 0.1039
epoch 7, loss: 0.0426
epoch 8, loss: 0.0186
epoch 9, loss: 0.1586


## Evaluating our model on the test data

Le réseau est tour à tour attaqué grâce aux méthodes FGSM puis PGD.

L'apprentissage adversarial est réalisé pour chaque attaque sur le jeu d'entraînement. Une fois l'apprentissage réalisé, l'appel de la fonction d'évaluation sur le jeu de test permet de calculer l'accuracy du réseau entrainé sur les données perturbées de ce jeu.

A noter :
- le critère et l'optimiseur sont déclarés une seule fois dan la mesure où ils sont identiques quel que soit le type d'attaque réalisé ;
- on instancie le réseau convolutif une première fois via `model = ConvModel()` pour l'attaque FGSM et une seconde fois via `model2 = ConvModel()` de sorte à ce que la seconde attaque (PGD) ne soit pas réalisée sur le modèle déjà entraîné lors de la première attaque (FGSM).  

In [9]:
# [CELL ID] 7

def eval_model(model, loader, attack=None):
  """Function to evaluate your model on a specific loader"""
  correct, n = 0, 0
  for imgs, labels in loader:
    imgs, labels = imgs.to(device), labels.to(device)
    
    if attack:
      imgs = attack.compute(imgs, labels)

    output = model(imgs)
    
    # transforming the output to probability by applying log_softmax function
    proba = F.log_softmax(output, dim=1)
    pred = proba.max(1, keepdim=True)[1]

    correct += torch.sum(pred.squeeze()==labels).item()
    n += pred.shape[0]

  return correct/n

### Evaluating under FGSM attack

In [10]:
# we evaluate the model trained under FGSM on the original test set
normal_acc = eval_model(fgsm_model, test_loader, attack=None)

# we evaluate the model trained under FGSM on the test set under FGSM attack
attack_acc = eval_model(fgsm_model, test_loader, attack=fgsm_attack)


print("accuracy FGSM model vs NO attack::", normal_acc)
print("accuracy FGSM model vs FGSM attack::", attack_acc)

accuracy FGSM model vs NO attack:: 0.9960333333333333
accuracy FGSM model vs FGSM attack:: 0.9727166666666667


### Evaluating under PGD attack

In [11]:
# we evaluate the model trained under PGD on the original test set
normal_acc = eval_model(pgd_model, test_loader, attack=None)

# we evaluate the model trained under PGD on the test set under PGD attack
attack_acc = eval_model(pgd_model, test_loader, attack=pgd_attack)

print("accuracy PGD model vs NO attack::", normal_acc)
print("accuracy PGD model vs PGD attack::", attack_acc)

accuracy PGD model vs NO attack:: 0.9751666666666666
accuracy PGD model vs PGD attack:: 0.9738666666666667


### Evaluating under cross-attack
Nous allons dans cette section évaluer l'impact d'une cross-attack, c'est-à-dire, évaluer la performance d'un modèle entrainé contre une attaque FGSM avec des images modifiées par PGD et vice-versa.


In [12]:
# we evaluate the model trained under FGSM on the test set under PGD attack
fgsm_attack_acc = eval_model(fgsm_model, test_loader, attack=pgd_attack)

# we evaluate the model trained under PGD on the test set under FGSM attack
pgd_attack_acc = eval_model(pgd_model, test_loader, attack=fgsm_attack)

print("accuracy FGSM model vs PGD attack:", fgsm_attack_acc)
print("accuracy PGD model vs FGSM attack:", pgd_attack_acc)

accuracy FGSM model vs PGD attack: 0.9960333333333333
accuracy PGD model vs FGSM attack: 0.9374166666666667


## Conclusion

Les niveaux d'accuracy sur les données d'origine sont de l'ordre de 99%. Nous avons vu dans le notebook 2 que dans le cas du modèle convolutif (celui utilisé dans ce Notebook) les niveaux d'accuracy baissaient à 75% pour l'attaque FGSM et à 85% pour l'attaque PGD. Avec entraînement adversarial, les niveaux d'accucary sont de 95.12% après une attaque FGSM et de 95.61% après une attage PGD. Même si ces niveaux restent inférieurs à ceux obtenus sur les données d'origine, la baisse constatée est faible, preuve que l'entraînement adversarial a permis d'augmenter de façon très significative la robustesse du modèle à une attaque.